In [18]:
from pathlib import Path
import optuna
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import f1_score

In [2]:
relpath_to_csv_folder = Path("../../datasets/CIC-IDS-2017/MachineLearningCSV/MachineLearningCVE")
df_train = pd.read_csv(relpath_to_csv_folder / "train.csv")
df_test = pd.read_csv(relpath_to_csv_folder / "test.csv")

### Encode Labels to numbers referring to the encoding in AlertNet training

In [8]:
labelnames2encodednums = {
    "BENIGN": "0",
    "Bot": "1",
    "DDoS": "2",
    "DoS": "3",
    "FTP-Patator": "4",
    "PortScan": "5",
    "SSH-Patator": "6",
    "Web": "7"
}
df_train = df_train.replace({"Label": labelnames2encodednums})
df_train["Label"] = df_train["Label"].astype("int64")
df_test = df_test.replace({"Label": labelnames2encodednums})
df_test["Label"] = df_test["Label"].astype("int64")

df_train.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,53,251,2,2,82,152,41,41,41.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
1,15837,2307203,1,5,1375,24,1375,1375,1375.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
2,53,357,2,2,90,206,45,45,45.0,0.0,...,32,0.0,0.0,0,0,0.0,0.0,0,0,0
3,62209,10051963,1,5,6,30,6,6,6.0,0.0,...,20,16498.0,0.0,16498,16498,10000000.0,0.0,10000000,10000000,0
4,58372,12009644,1,6,1375,30,1375,1375,1375.0,0.0,...,20,22971.0,0.0,22971,22971,12000000.0,0.0,12000000,12000000,0


### Split data into features and labels

In [4]:
train_data = df_train.drop("Label", axis=1).values
train_labels = df_train["Label"].values
test_data = df_test.drop("Label", axis=1).values
test_labels = df_test["Label"].values

### Define evaluation function for the optimization by optuna

In [5]:
def objective(trial, train_data, train_labels, test_data, test_labels):
    param = {}
    param['n_estimators'] = trial.suggest_int('n_estimators', 1, 100, log=True)
    param['max_depth'] = trial.suggest_int('max_depth', 1, 100, log=True)

    model = RandomForestClassifier(**param)
    model.fit(train_data, train_labels)
    
    preds =  model.predict(test_data)
    score = mae(preds, test_labels)

    return score

In [6]:
EXECUTE_OPTIMIZATION = True

if EXECUTE_OPTIMIZATION:
    study = optuna.create_study()
    # start optimization
    study.optimize(lambda trial: objective(trial, train_data, train_labels, test_data, test_labels), n_trials=100, show_progress_bar=True)
    # print the best parameter
    print(f'Best parameter:{study.best_params}')

    # initialize RandomForestClassifier with the best parameter
    model = RandomForestClassifier(**study.best_params)
    model.fit(train_data, train_labels)

# Best parameter:{'n_estimators': 52, 'max_depth': 59}

[I 2022-07-04 20:32:49,649] A new study created in memory with name: no-name-777d8ca4-0923-434e-ad6d-4cd109b8671f
/home/toda/.venv/ml/lib/python3.8/site-packages/optuna/progress_bar.py:47: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()
  1%|          | 1/100 [00:00<00:32,  3.04it/s]

[I 2022-07-04 20:32:49,984] Trial 0 finished with value: 0.009128892946174643 and parameters: {'n_estimators': 2, 'max_depth': 32}. Best is trial 0 with value: 0.009128892946174643.


  3%|▎         | 3/100 [00:02<01:10,  1.37it/s]

[I 2022-07-04 20:32:51,783] Trial 1 finished with value: 0.6846669709630983 and parameters: {'n_estimators': 42, 'max_depth': 2}. Best is trial 0 with value: 0.009128892946174643.
[I 2022-07-04 20:32:51,962] Trial 2 finished with value: 1.083529370457498 and parameters: {'n_estimators': 6, 'max_depth': 1}. Best is trial 0 with value: 0.009128892946174643.


  4%|▍         | 4/100 [00:06<03:36,  2.25s/it]

...

 99%|█████████▉| 99/100 [10:41<00:07,  7.88s/it]

[I 2022-07-04 20:43:31,186] Trial 98 finished with value: 0.005477335767704786 and parameters: {'n_estimators': 100, 'max_depth': 32}. Best is trial 73 with value: 0.004072890699062533.


100%|██████████| 100/100 [10:45<00:00,  6.46s/it]


[I 2022-07-04 20:43:35,349] Trial 99 finished with value: 0.00533689126084056 and parameters: {'n_estimators': 29, 'max_depth': 27}. Best is trial 73 with value: 0.004072890699062533.
Best parameter:{'n_estimators': 52, 'max_depth': 59}


RandomForestClassifier(max_depth=59, n_estimators=52)

In [20]:
preds =  model.predict(test_data)
f1score = f1_score(preds, test_labels, average="weighted")
print(f"F1 score of optimized model: {f1score}")

F1 score of optimized model: 0.9970387875278569


In [15]:
df = pd.DataFrame()
df["FeatureName"] = df_train.drop("Label", axis=1).columns
df["Importance"] = model.feature_importances_
df.sort_values("Importance", ascending=False).head(n=20)

,FeatureName,Importance
0,Destination Port,0.092784
63,Subflow Fwd Bytes,0.038117
65,Subflow Bwd Bytes,0.035389
53,Avg Fwd Segment Size,0.034009
12,Bwd Packet Length Mean,0.032598
6,Fwd Packet Length Max,0.031366
4,Total Length of Fwd Packets,0.031111
69,min_seg_size_forward,0.030677
66,Init_Win_bytes_forward,0.029622
34,Fwd Header Length,0.029445
